In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

In [2]:
""" 数据抽查 sz_max_50 """
def datacheck():
    sum = len(df)
    bins = [-float('inf'), 0, 3, 8, 15, float('inf')] 
    labels = ['0-0', '0-3%', '3-8%',  '8-15%', '>15%']

    df['sz_max_5_bin'] = pd.cut(df['sz_max_5'], bins=bins, labels=labels)
    print( df['sz_max_5_bin'].value_counts()/sum, df['sz_max_5'].mean() )

    df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
    print( df['sz_max_10_bin'].value_counts()/sum, df['sz_max_10'].mean() )

    df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
    print( df['sz_max_20_bin'].value_counts()/sum, df['sz_max_20'].mean() )


In [7]:

# 指标库读出基本数据 
# xl_01dgree = 0.00174533 # 0.1度 
# xl_02dgree = 0.00349067 # 0.2度 
# xl_03dgree = 0.00523604 # 0.3度
# xl_dgree = xl_03dgree
# print( 'd60 斜率：', str(xl_dgree) )

# trade_date like '%2023-06%' and \
sql = "SELECT * \
from day_zhibiao \
where \
jx_dg_250>-0.1 and jx_dg_60>-0.1 and \
jx_days_ud60>=20 and jx_days_ud60<=40 and \
jx_dg_20>=0 and jx_zs_20>=1 and pct_fl_3_10>1.5 and \
(pct_fl_1_3>1.5 or k_red>=1) "
# (pct_fl_3_10>2 or k_red>=1) "

print( sql )
df = pd.read_sql_query(sql, conn)

print('shape: ', df.shape, '\n')
datacheck()

SELECT * from day_zhibiao where jx_dg_250>-0.1 and jx_dg_60>-0.1 and jx_days_ud60>=20 and jx_days_ud60<=40 and jx_dg_20>=0 and jx_zs_20>=1 and pct_fl_3_10>1.5 and (pct_fl_1_3>1.5 or k_red>=1) 
shape:  (6680, 81) 

0-3%     0.364820
0-0      0.286976
3-8%     0.239072
8-15%    0.079790
>15%     0.029341
Name: sz_max_5_bin, dtype: float64 3.248714422799401
3-8%     0.295659
0-3%     0.276497
0-0      0.212725
8-15%    0.143263
>15%     0.071856
Name: sz_max_10_bin, dtype: float64 5.322344039610779
3-8%     0.292365
0-3%     0.192515
8-15%    0.190269
0-0      0.163922
>15%     0.160928
Name: sz_max_20_bin, dtype: float64 8.394864690673652


In [14]:
# 检测数据
col_check = ['symbol', 'trade_date', 'pct_change', 'sz_max_10', \
    'diff', 'dea', \
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_20_60', \
    'sz_max_5','sz_max_20',
     ]
sorting = df.sort_values(['trade_date', 'symbol'], ascending=False)
sorting[df['trade_date']>'2023-07'][col_check]
# sorting.head(5)[col_check]

d:\temp\ipykernel_16692\673781611.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sorting[df['trade_date']>'2023-07'][col_check]


,symbol,trade_date,pct_change,sz_max_10,diff,dea,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,jx_dg_5,jx_zs_5,jx_zs_10,jx_zs_20,jx_zs_60,pct_fl_1_3,pct_fl_3_10,pct_fl_20_60,sz_max_5,sz_max_20
1904,601326,2023-07-31,0.59,0.00000,0.004450,-0.016492,22,0.099766,-0.060921,0.069735,...,0.444940,6,6,6,0,0.992921,2.25076,0.286992,0.000000,0.00000
868,300052,2023-07-31,1.50,0.00000,-0.229187,-0.358734,27,-0.033369,-0.092354,0.000000,...,0.737673,5,3,3,0,0.935267,3.18792,0.344795,0.000000,0.00000
109,000617,2023-07-31,0.82,0.00000,-0.031153,-0.092951,28,0.088363,-0.026599,0.048635,...,0.922763,5,5,2,0,1.183510,2.46897,0.351347,0.000000,0.00000
1931,601628,2023-07-27,2.07,0.00000,-0.706987,-0.913833,27,0.013772,-0.092188,-0.019578,...,0.861108,3,3,2,0,0.796837,2.48075,0.973793,10.240800,0.00000
455,002323,2023-07-26,0.35,0.00000,-0.011240,-0.024106,34,-0.069178,-0.090425,0.041384,...,0.622305,4,3,2,0,0.914567,2.04660,0.492178,0.704225,0.00000
1815,600917,2023-07-24,0.15,0.00000,-0.006393,-0.022311,40,-0.071423,-0.027315,0.046085,...,0.033438,2,2,19,0,0.444982,2.10422,0.836334,3.488370,0.00000
1534,600327,2023-07-21,0.85,2.95983,-0.008487,-0.026502,28,0.014201,0.032266,0.092942,...,0.344115,4,4,4,0,1.044270,2.13400,0.545167,1.479920,0.00000
1266,300783,2023-07-21,3.80,3.86330,-0.249146,-0.382205,20,0.000000,0.065321,-0.059385,...,0.641695,7,3,3,0,2.124210,2.50632,0.494225,2.278360,0.00000
277,001322,2023-07-20,2.97,13.73340,-0.237452,-0.336983,23,-0.041305,-0.081080,0.097994,...,0.661951,4,3,2,0,2.174280,2.69994,1.260050,2.192730,0.00000
382,002195,2023-07-13,1.01,1.33779,-0.024750,-0.034614,26,0.037917,-0.098852,-0.078380,...,0.554330,3,2,2,0,0.846044,2.72602,0.679711,0.000000,0.00000


In [5]:
""" 数据抽查 sz_max_5 """
sum = len(df)
bins = [-float('inf'), 0, 5, 8, float('inf')] 
labels = ['0-0', '0-5%', '5-8%', '>8%']
df['sz_max_5_bin'] = pd.cut(df['sz_max_5'], bins=bins, labels=labels)
# print( df['sz_max_10_bin'].value_counts() )
df['sz_max_5_bin'].value_counts()/sum, df['sz_max_5'].mean()

(0-5%    0.455551
 0-0     0.311809
 >8%     0.122954
 5-8%    0.109686
 Name: sz_max_5_bin, dtype: float64,
 3.3890430066342323)

In [6]:
""" 数据抽查 sz_max_10 """
# sum = len(df)
# bins = [-float('inf'), 0,3,8, float('inf')] 
# labels = ['0', '0-3', '3-8', '>8']
# df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
# df['sz_max_10_bin'].value_counts(), df['sz_max_10_bin'].value_counts()/sum

' 数据抽查 sz_max_10 '

In [7]:
""" 数据抽查 sz_max_20 """
sum = len(df)
bins = [-float('inf'), 0, 5, 12, float('inf')] 
labels = ['<=0', '0-5','5-12', '>12']
df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
df['sz_max_20_bin'].value_counts(), df['sz_max_20_bin'].value_counts()/sum, df['sz_max_20'].mean()

(0-5     666
 5-12    643
 >12     581
 <=0     371
 Name: sz_max_20_bin, dtype: int64,
 0-5     0.294560
 5-12    0.284387
 >12     0.256966
 <=0     0.164087
 Name: sz_max_20_bin, dtype: float64,
 9.157768853206546)

In [8]:
# 涨幅阶梯化

def jieti_5(df):
    # rate = 0, 1, 2, 3
    if df['sz_max_5']==0: rate = 0        
    if df['sz_max_5']>0 and df['sz_max_5']<=3: rate = 1          
    if df['sz_max_5']>3 and df['sz_max_5']<=8: rate = 2          
    if df['sz_max_5']>8: rate = 3        
    return rate
df['sz_jt_5'] = df.apply(jieti_5, axis=1) 

def jieti_10(df):
    # rate = 0, 1, 2, 3
    if df['sz_max_10']==0: rate = 0        
    if df['sz_max_10']>0 and df['sz_max_10']<=3: rate = 1          
    if df['sz_max_10']>3 and df['sz_max_10']<=8: rate = 2          
    if df['sz_max_10']>8: rate = 3
    return rate
df['sz_jt_10'] = df.apply(jieti_10, axis=1) 

def jieti_20(df):
    # rate = 0, 1, 2
    if df['sz_max_20'] <= 8: rate = 0        
    if df['sz_max_20']>8 and df['sz_max_10']<=15: rate = 1        
    if df['sz_max_20']>15: rate = 2        
    return rate
df['sz_jt_20'] = df.apply(jieti_20, axis=1) 

In [9]:
# 数据监测
df[ ['sz_max_5', 'sz_jt_5', 'sz_max_10','sz_jt_10', 'sz_max_20', 'sz_jt_20']].head(20)
# df[ ['sz_max_20', 'sz_jt_20']].head(20)

,sz_max_5,sz_jt_5,sz_max_10,sz_jt_10,sz_max_20,sz_jt_20
0,1.280620,1,1.280620,1,1.280620,0
1,1.337050,1,1.337050,1,1.337050,0
2,2.720400,1,2.720400,1,3.274560,0
3,3.389830,2,25.988700,3,25.988700,2
4,0.000000,0,0.000000,0,3.309690,0
5,0.634921,1,0.634921,1,3.968250,0
6,1.637550,1,3.930130,2,12.554600,1
7,0.000000,0,0.000000,0,0.000000,0
8,0.000000,0,0.000000,0,0.000000,0
9,1.240310,1,1.240310,1,2.945740,0


In [10]:
""" 形成数据集 """

# 0-1类训练目标s：
df['test00'] = df['sz_max_20'] > 10
df['test5_8'] = df['sz_max_5'] > 8
df['test10_10'] = df['sz_max_10'] > 10
df['test20_10'] = df['sz_max_20'] > 10
df['test20_20'] = df['sz_max_20'] > 20
df['test20_30'] = df['sz_max_20'] > 30
# 多分类训练目标
df['class0'] = df['sz_jt_10']
# 确定输出列  
col_list_ds = ['ts_code', 'symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
    'class0','sz_jt_5','sz_jt_10','sz_jt_20' ]
df_ds = df[col_list_ds].copy()
# 'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
# df_ds[ df_ds['test00'] ].shape, df_ds[ df_ds['test5_8'] ].shape, df_ds[ df_ds['test10_10'] ].shape, df_ds[df_ds['test20_10']].shape, df_ds[df_ds['test20_20']].shape, df_ds[ df_ds['test20_30'] ].shape

In [11]:
""" 输出到mysql """
# 删除旧数据
sql = "truncate table ds_qf60"
basic.exec_sql(conn, sql)
# 输出新数据
df_ds.to_sql('ds_qf60', con=conn, index=False, if_exists='append', chunksize=2000) 

2261

In [12]:
# name: 数据库中的表名
# con: 与read_sql中相同，数据库连接的驱动
# if_exits： 当数据库中的这个表存在的时候，采取的措施是什么，包括三个值，默认为fail
# fail，若表存在，则不进行数据表写入的相关操作
# replace：若表存在，将数据库表中的数据覆盖；
# append：若表存在，将数据写到原表的后面。
# index：是否将df的index单独写到一列中
# index_label:指定列作为df的index输出，此时index为True
# dtype: 指定列的输出到数据库中的数据类型。字典形式储存：{column_name: sql_dtype}